In [11]:
import nest_asyncio
nest_asyncio.apply()

In [12]:
import os
from dotenv import load_dotenv
import rich
from agents import Agent, Runner, GuardrailFunctionOutput, InputGuardrail
from pydantic import BaseModel
# import asyncio

In [13]:
load_dotenv()

api_key = os.getenv('OPENAI_API_KEY')
MODEL = "x"
# print(api_key)

In [14]:
class HomeworkOutput(BaseModel):
    is_homework: bool
    reasoning: str

In [15]:
math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description = "Specialist agent for math questions",
    instructions = "You provide help with math problems. Explain your reasoning at each step and include examples.",
    model=MODEL
    
)

history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description = "Specialist agent for historical questions",
    instructions = "You provide assistance with historical queries. Explain import events and context clearly.",
    model=MODEL
)

In [16]:
guardrail_agent = Agent(
    name="Guardrail Check",
    instructions = "Check if the user is asking about homework",
    output_type = HomeworkOutput,
    # model=MODEL
)

In [17]:
async def homework_guardrail(ctx, agent, input_data):
    result = await Runner.run(guardrail_agent, input_data, context=ctx.context)
    final_output = result.final_output_as(HomeworkOutput)
    rich.print(result)
    rich.print(final_output)
    return GuardrailFunctionOutput(
        output_info = final_output,
        tripwire_triggered = not final_output.is_homework
    )

Handoffs

In [18]:
triage_agent = Agent(
    name="Triage Agent",
    instructions = "You determine which agent to use based on the user's homework question.",
    handoffs = [math_tutor_agent, history_tutor_agent],
    input_guardrails= [
        InputGuardrail(guardrail_function=homework_guardrail)
    ],
    model=MODEL
)

Run the agent Orchestration

In [19]:
result = await Runner.run(
    triage_agent,
    "who was the first president of the united states?"
    # "Who won the first world war?"
)
print("result at triage agent")
rich.print(result)
rich.print(result.final_output)

RunResult(
    input='who was the first president of the united states?',
    new_items=[
        MessageOutputItem(
            agent=Agent(
                name='Guardrail Check',
                instructions='Check if the user is asking about homework',
                handoff_description=None,
                handoffs=[],
                model=None,
                model_settings=ModelSettings(
                    temperature=None,
                    top_p=None,
                    frequency_penalty=None,
                    presence_penalty=None,
                    tool_choice=None,
                    parallel_tool_calls=None,
                    truncation=None,
                    max_tokens=None,
                    reasoning=None,
                    metadata=None,
                    store=None
                ),
                tools=[],
                mcp_servers=[],
                mcp_config={},
                input_guardrails=[],
                output_guardrails=[],
                output_type=<class '__main__.HomeworkOutput'>,
                hooks=None,
                tool_use_behavior='run_llm_again',
                reset_tool_choice=True
            ),
            raw_item=ResponseOutputMessage(
                id='msg_67fcb5546c748192a30ff7b5f85c243f02f3d4df48fe0cf1',
                content=[
                    ResponseOutputText(
                        annotations=[],
                        text='{"is_homework":true,"reasoning":"The question is straightforward and factual, 
commonly encountered in educational contexts, particularly in history classes."}',
                        type='output_text'
                    )
                ],
                role='assistant',
                status='completed',
                type='message'
            ),
            type='message_output_item'
        )
    ],
    raw_responses=[
        ModelResponse(
            output=[
                ResponseOutputMessage(
                    id='msg_67fcb5546c748192a30ff7b5f85c243f02f3d4df48fe0cf1',
                    content=[
                        ResponseOutputText(
                            annotations=[],
                            text='{"is_homework":true,"reasoning":"The question is straightforward and factual, 
commonly encountered in educational contexts, particularly in history classes."}',
                            type='output_text'
                        )
                    ],
                    role='assistant',
                    status='completed',
                    type='message'
                )
            ],
            usage=Usage(requests=1, input_tokens=77, output_tokens=30, total_tokens=107),
            referenceable_id='resp_67fcb55403dc8192a66d76bfdc5370c702f3d4df48fe0cf1'
        )
    ],
    final_output=HomeworkOutput(
        is_homework=True,
        reasoning='The question is straightforward and factual, commonly encountered in educational contexts, 
particularly in history classes.'
    ),
    input_guardrail_results=[],
    output_guardrail_results=[],
    _last_agent=Agent(
        name='Guardrail Check',
        instructions='Check if the user is asking about homework',
        handoff_description=None,
        handoffs=[],
        model=None,
        model_settings=ModelSettings(
            temperature=None,
            top_p=None,
            frequency_penalty=None,
            presence_penalty=None,
            tool_choice=None,
            parallel_tool_calls=None,
            truncation=None,
            max_tokens=None,
            reasoning=None,
            metadata=None,
            store=None
        ),
        tools=[],
        mcp_servers=[],
        mcp_config={},
        input_guardrails=[],
        output_guardrails=[],
        output_type=<class '__main__.HomeworkOutput'>,
        hooks=None,
        tool_use_behavior='run_llm_again',
        reset_tool_choice=True
    )
)

HomeworkOutput(
    is_homework=True,
    reasoning='The question is straightforward and factual, commonly encountered in educational contexts, 
particularly in history classes.'
)

result at triage agent


RunResult(
    input='who was the first president of the united states?',
    new_items=[
        HandoffCallItem(
            agent=Agent(
                name='Triage Agent',
                instructions="You determine which agent to use based on the user's homework question.",
                handoff_description=None,
                handoffs=[
                    Agent(
                        name='Math Tutor',
                        instructions='You provide help with math problems. Explain your reasoning at each step and 
include examples.',
                        handoff_description='Specialist agent for math questions',
                        handoffs=[],
                        model='gpt-4o-mini',
                        model_settings=ModelSettings(
                            temperature=None,
                            top_p=None,
                            frequency_penalty=None,
                            presence_penalty=None,
                            tool_choice=None,
                            parallel_tool_calls=None,
                            truncation=None,
                            max_tokens=None,
                            reasoning=None,
                            metadata=None,
                            store=None
                        ),
                        tools=[],
                        mcp_servers=[],
                        mcp_config={},
                        input_guardrails=[],
                        output_guardrails=[],
                        output_type=None,
                        hooks=None,
                        tool_use_behavior='run_llm_again',
                        reset_tool_choice=True
                    ),
                    Agent(
                        name='History Tutor',
                        instructions='You provide assistance with historical queries. Explain import events and 
context clearly.',
                        handoff_description='Specialist agent for historical questions',
                        handoffs=[],
                        model='gpt-4o-mini',
                        model_settings=ModelSettings(
                            temperature=None,
                            top_p=None,
                            frequency_penalty=None,
                            presence_penalty=None,
                            tool_choice=None,
                            parallel_tool_calls=None,
                            truncation=None,
                            max_tokens=None,
                            reasoning=None,
                            metadata=None,
                            store=None
                        ),
                        tools=[],
                        mcp_servers=[],
                        mcp_config={},
                        input_guardrails=[],
                        output_guardrails=[],
                        output_type=None,
                        hooks=None,
                        tool_use_behavior='run_llm_again',
                        reset_tool_choice=True
                    )
                ],
                model='gpt-4o-mini',
                model_settings=ModelSettings(
                    temperature=None,
                    top_p=None,
                    frequency_penalty=None,
                    presence_penalty=None,
                    tool_choice=None,
                    parallel_tool_calls=None,
                    truncation=None,
                    max_tokens=None,
                    reasoning=None,
                    metadata=None,
                    store=None
                ),
                tools=[],
                mcp_servers=[],
                mcp_config={},
                input_guardrails=[
                    InputGuardrail(
                        guardrail_function=<function homework_guardrail at 0x000002AB844ACC20>,
                        name=None
                    )
                ],
                output_guardrails=[],
   

The first president of the United States was George Washington. He served from April 30, 1789, to March 4, 1797. 
Washington was a pivotal figure in the American Revolutionary War, leading the Continental Army to victory against 
British forces. He was unanimously elected as president and is often referred to as the "Father of His Country" for
his leading role in the founding of the nation.

Washington set many precedents for the presidency and helped to shape the new government. He emphasized the 
importance of a strong union and was wary of political parties and foreign alliances, which he warned against in 
his farewell address. His leadership and vision laid the groundwork for the future of the United States.

In [20]:
rich.print(result.to_input_list())

[
    {'content': 'who was the first president of the united states?', 'role': 'user'},
    {
        'arguments': '{}',
        'call_id': 'call_f2At74CUkCCS8QVZjcFlOhFp',
        'name': 'transfer_to_history_tutor',
        'type': 'function_call',
        'id': 'fc_67fcb554d27c819285cd331a9d0f26e10b651a276b523530',
        'status': 'completed'
    },
    {
        'call_id': 'call_f2At74CUkCCS8QVZjcFlOhFp',
        'output': "{'assistant': 'History Tutor'}",
        'type': 'function_call_output'
    },
    {
        'id': 'msg_67fcb55629bc81929ce7bc96adadda240b651a276b523530',
        'content': [
            {
                'annotations': [],
                'text': 'The first president of the United States was George Washington. He served from April 30, 
1789, to March 4, 1797. Washington was a pivotal figure in the American Revolutionary War, leading the Continental 
Army to victory against British forces. He was unanimously elected as president and is often referred to as the 
"Father of His Country" for his leading role in the founding of the nation.\n\nWashington set many precedents for 
the presidency and helped to shape the new government. He emphasized the importance of a strong union and was wary 
of political parties and foreign alliances, which he warned against in his farewell address. His leadership and 
vision laid the groundwork for the future of the United States.',
                'type': 'output_text'
            }
        ],
        'role': 'assistant',
        'status': 'completed',
        'type': 'message'
    }
]

In [10]:
result = await Runner.run(
    triage_agent,
    "what is life"
)
print(result.final_output)

HomeworkOutput(
    is_homework=False,
    reasoning="The question 'what is life' is a philosophical or existential inquiry rather than a homework 
question. It often relates to discussions about the nature, purpose, and value of existence."
)

InputGuardrailTripwireTriggered: Guardrail InputGuardrail triggered tripwire